# `tensor` 탐구


## 텐서의 속성

- `shape`
- `dtype`
- `device`
- `layout`
- `memory_format`


### `shape`


### `dtype`


In [2]:
import torch

float_tensor1 = torch.tensor([1, 2, 3], dtype=torch.float)
float_tensor2 = torch.tensor([4, 5, 6], dtype=torch.float32)
float_tensor3 = torch.FloatTensor([7, 8, 9])  # Legacy Constructors

for tensor in [float_tensor1, float_tensor2, float_tensor3]:
    print(f"{tensor}'s element data type is {tensor.dtype}")


/home/minhyuk/setup/miniconda3/envs/pytorch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor([1., 2., 3.])'s element data type is torch.float32
tensor([4., 5., 6.])'s element data type is torch.float32
tensor([7., 8., 9.])'s element data type is torch.float32


In [162]:
# 32-bit floating point
print(torch.float)
# 32-bit integer (signed)
print(torch.int)
# Boolean
print(torch.bool)

# 64-bit floating point
print(torch.double)
# 64-bit inteber (signed)
print(torch.long)


torch.float32
torch.int32
torch.bool
torch.float64
torch.int64


In [15]:
torch.tensor([1, 2, 3]).dtype


ValueError: only one element tensors can be converted to Python scalars

### `device`

- cpu  
  `"cpu"`
- gpu  
  `"cuda"`


In [167]:
cpu_tensor = torch.tensor([1, 2, 3])
gpu_tensor1 = torch.tensor([4, 5, 6], device="cuda")
gpu_tensor2 = torch.tensor([7, 8, 9], device=torch.device("cuda"))

for tensor in [cpu_tensor, gpu_tensor1, gpu_tensor2]:
    print(f"{tensor}'s device is {tensor.device}")


tensor([1, 2, 3])'s device is cpu
tensor([4, 5, 6], device='cuda:0')'s device is cuda:0
tensor([7, 8, 9], device='cuda:0')'s device is cuda:0


In [179]:
available_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(available_device)


cuda


### `layout`

아직 베타 버전, 바뀔 수 있음

> The `torch.layout` class is in beta and subject to change.

- `torch.strided`: Dense Tensor
- `torch.sparse_coo`: Sparse Tensor


### `memory_format`

- `torch.contiguous_format`
- `torch.channels_last`
- `torch.preserve_format`


## 텐서 생성하기

- `torch.tensor()`  
  인자로 전달된 array-like 데이터로 새로운 텐서를 반환한다.   
- `torch.XXX()`  
  인자로 전달된 size의 텐서를 새로 만든다.   
- `torch.XXX_like()`  
  인자로 전달된 텐서와 같은 size의 텐서를 새로 만든다.   
  `dtype`, `layout`, `device`를 따로 설정하지 않으면 인자로 전달된 텐서와 같은 것을 따른다.   
- `self.new_XXX()`  
  인자로 size를 전달한다.  
  `dtype`과 `device`를 유지하는 대신 새로운 텐서를 만든다.


In [19]:
import torch

data1 = torch.Tensor([1, 2, 3, 4])
data2 = data1.new_zeros((3, 2))
print(data1)
print(data2)

tensor([1., 2., 3., 4.])
tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])


Legacy Constructor

- `torch.FloatTensor()`
- `torch.cuda.FloatTensor()`


종류
* `full`   
매개변수 `size`와 `fill_value`
* `empty`   
매개변수 `size`
* `ones`   
매개변수 `size`   
* `zeros`   
매개변수 `size`   

* `torch.tensor()`   
항상 새롭게 복사된다. (deep copy)
* `torch.asarray()` or `torch.as_tensor()`   
조건에 따라서 복사될 수도, 메모리를 공유할 수도 있다.   
`dtype`, `device`등을 새로 설정할 수 있다. (이럴 때 복사된다.)
* `torch.from_numpy()`   
항상 `ndarray`와 같은 메모리를 공유한다.(shallow copy)   
`dtype`, `device`등을 바꿀 수 없다.

In [29]:
import torch
import numpy as np
numpy_data = np.zeros((3, 4), dtype=float)
tensor_data = torch.from_numpy(numpy_data)
tensor_data.dtype

torch.float64

In [44]:
print(torch.arange(0, 10))
print(torch.linspace(0, 1, 11))
print(torch.logspace(0, 1, 11))

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
tensor([0.0000, 0.1000, 0.2000, 0.3000, 0.4000, 0.5000, 0.6000, 0.7000, 0.8000,
        0.9000, 1.0000])
tensor([1.0000e+00, 3.1623e+00, 1.0000e+01, 3.1623e+01, 1.0000e+02, 3.1623e+02,
        1.0000e+03, 3.1623e+03, 1.0000e+04, 3.1623e+04, 1.0000e+05])


In [48]:
torch.eye(4)

tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])

## 연산

`self.XXX`는 연산 결과가 반환되지만 self는 반영되지 않는다.  
`self.XXX_`는 연산 결과가 in-place로 적용되고 반환도 된다.


### inplace


In [4]:
data = torch.zeros(2, 3)
print("===Before===")
print(data)
data.add_(12)
print("===After===")
print(data)


===Before===
tensor([[0., 0., 0.],
        [0., 0., 0.]])
===After===
tensor([[12., 12., 12.],
        [12., 12., 12.]])


## View

### Reshape

`view`  
반드시 같은 데이터를 공유한다. (shallow copy)  
조건에 따라서 RuntimeError을 띄우기도 한다.

`reshape`  
같은 데이터를 공유할 때도 있지만, 조건에 따라서 복사해서 제공할 때도 있다.  
그러나 Copying과 viewing이 되는 상황을 구분해서 사용하는 것은 좋지 않다.

> but you should not depend on the copying vs. viewing behavior.


In [35]:
data = torch.arange(0, 24).reshape(1, 2, 3, 4)
viewed_data = data.view(1, 3, 2, 4)
reshaped_data = data.view(1, 3, 2, 4)

for tensor in [data, viewed_data, reshaped_data]:
    print("shape:", tensor.shape)


shape: torch.Size([1, 2, 3, 4])
shape: torch.Size([1, 3, 2, 4])
shape: torch.Size([1, 3, 2, 4])


In [36]:
tranposed_data = torch.arange(0, 24).reshape(1, 2, 3, 4).T
try:
    viewed_data = tranposed_data.view(1, 3, 2, 4)
except RuntimeError as error:
    print(error)
reshaped_data = tranposed_data.reshape(1, 3, 2, 4)

reshaped_data.add_(10)
for tensor in [tranposed_data, reshaped_data]:
    print("shape:", tensor.shape)


view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.
shape: torch.Size([4, 3, 2, 1])
shape: torch.Size([1, 3, 2, 4])


In [83]:
data1 = torch.arange(-2, 3)
data2 = torch.zeros_like(data1)
data1.argwhere()
torch.where(data1 > 0, data1, data2)
data1.nonzero()

(tensor([0, 1, 3, 4]),)

`contiguous`


### Dimension swap

- `transpose()`또는 `swapdims()`, `swapaxes()`  
  두 차원끼리 교환할 때 사용한다. (shallow copy)
- `permute()`  
  원하는 차원들을 교환할 때 사용한다. (shallow copy)
- `T`  
  `permute(n-1, n-2, ..., 0)`과 동일하다.
- `view()` and `reshape()`
- `moveaxis` 또는 `movedim()`


### Dimension insert

* `unsqueeze`  
* `view` and `reshape`  
* `expand`


### Dimension reduction

* `ravel`  
* `flatten`
* `squeeze`   
size가 1인 차원을 제거한다.   
입력된 차원의 size가 1이면 그 차원을 제거한다.


### Concatenate

* `torch.cat()` 또는 `torch.concat()`   
붙이려는 차원의 수가 일치해야 한다.
* `torch.stack()`
* `vstack` and `row_stack`   
Stack tensors in sequence vertically (row wise).
* `hstack`   
Stack tensors in sequence horizontally (column wise).
* `column_stack`   
Creates a new tensor by horizontally stacking the tensors in tensors.
* `dstack`   
Stack tensors in sequence depthwise (along third axis).   

In [55]:
data1 = torch.tensor([1, 2, 3])
data2 = torch.tensor([4, 5, 6])
torch.dstack((data1, data2)).shape

torch.Size([1, 3, 2])

### Split
* `tensor_split`   
정수거나 스칼라면 입력된 인자만큼의 section개로 나눈다.   
정수형의 list, tuple 또는 1차원 tensor면 차원에 따른 indices로 나눈다.   
* `chunk()`
* `dsplit`
* `hsplit`
* `split`
* `vsplit`
* `unbind`   
Returns a tuple of all slices along a given dimension, already without it.

## 다루지 않은 것

- Complex Type 관련된 것  
  `torch.complex32`, `Tensor.H` 등
- polar 등 다양한 타입
- scatter

In [75]:
data = torch.tensor([[1, 2], [3,4]])
data.tile((2, 2))

tensor([[1, 2, 1, 2],
        [3, 4, 3, 4],
        [1, 2, 1, 2],
        [3, 4, 3, 4]])

In [92]:
data = torch.arange(0, 12).reshape(2, 2, 3)
# torch.take()
# torch.take_along_dim()
torch.select(data, 1, 0)  # data[:, 0, :]
# torch.narrow()
torch.masked_select(data, data > 5)
torch.index_select(data, 2, torch.tensor([0, 2]))
# index_add
# gather

tensor([[[ 0,  2],
         [ 3,  5]],

        [[ 6,  8],
         [ 9, 11]]])

## Reduction


## Comparison